In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize
import time
from tqdm.notebook import tqdm
import datetime
import os
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
path = r'E:\competitions\kaggle\magnit_recsys-in-practice\data\jester_dataset_1_joke_texts\jokes'
ff = glob.glob(path + r'\*.html')
ff

['E:\\competitions\\kaggle\\magnit_recsys-in-practice\\data\\jester_dataset_1_joke_texts\\jokes\\init1.html',
 'E:\\competitions\\kaggle\\magnit_recsys-in-practice\\data\\jester_dataset_1_joke_texts\\jokes\\init10.html',
 'E:\\competitions\\kaggle\\magnit_recsys-in-practice\\data\\jester_dataset_1_joke_texts\\jokes\\init100.html',
 'E:\\competitions\\kaggle\\magnit_recsys-in-practice\\data\\jester_dataset_1_joke_texts\\jokes\\init11.html',
 'E:\\competitions\\kaggle\\magnit_recsys-in-practice\\data\\jester_dataset_1_joke_texts\\jokes\\init12.html',
 'E:\\competitions\\kaggle\\magnit_recsys-in-practice\\data\\jester_dataset_1_joke_texts\\jokes\\init13.html',
 'E:\\competitions\\kaggle\\magnit_recsys-in-practice\\data\\jester_dataset_1_joke_texts\\jokes\\init14.html',
 'E:\\competitions\\kaggle\\magnit_recsys-in-practice\\data\\jester_dataset_1_joke_texts\\jokes\\init15.html',
 'E:\\competitions\\kaggle\\magnit_recsys-in-practice\\data\\jester_dataset_1_joke_texts\\jokes\\init16.html',
 

In [3]:
jokes = []

for filename in ff:
    with open(filename, 'rt') as file:
        rows = file.readlines()

    start_index = None
    end_index = None
    for i, row in enumerate(rows):
        if 'begin of joke' in row:
            start_index = i

        if 'end of joke' in row:
            joke = rows[start_index + 1 : i]

            jokes.append(''.join(joke)
                         .replace('<P>', '')
                         .replace('<p>', '')
                         .replace('</i>', ' ')
                         .replace('<i>', ' ')
                         .replace('<BR>', ' ')
                         .replace('&nbsp;', ' ')
                         .replace('<br>', ' ')
                         .replace('</UL>', ' ')
                         .replace('\n', ' ')
                        )

            continue

In [4]:
len(jokes)

100

In [5]:
for j in jokes:
    print(j)

A man visits the doctor. The doctor says "I have bad news for you.You have cancer and Alzheimer's disease".  The man replies "Well,thank God I don't have cancer!" 
Two cannibals are eating a clown, one turns to other and says:  "Does this taste funny to you?   
Q: Whats the difference between greeting a Queen and greeting the President of the United  States?  A: You only have to get on  one knee  to greet the queen. 
Q. What do a hurricane, a tornado, and a redneck divorce all have in common?  A. Someone's going to lose their trailer... 
A guy stood over his tee shot for what seemed an eternity, looking up, looking down, measuring the distance, figuring the wind direction and speed. Driving his partner nuts.  Finally his exasperated partner says, "What the hell is taking so long? Hit the goddamn ball!" The guy answers, "My wife is up there watching me from the clubhouse. I want to make this a perfect shot." "Well, hell, man, you don't stand a snowball's chance in hell of hitting her fr

In [6]:
import collections

import gensim.utils
JokeDocument = collections.namedtuple('JokeDocument', 'words tags')

In [7]:
alldocs = []
for i, joke in enumerate(jokes):    
    tokens = gensim.utils.to_unicode(joke).split()
    alldocs.append(JokeDocument(tokens, [i]))

In [8]:
alldocs[27]

JokeDocument(words=['What', 'do', 'you', 'call', 'an', 'American', 'in', 'the', 'finals', 'of', 'the', 'world', 'cup?', '"Hey', 'Beer', 'Man!"'], tags=[27])

In [9]:
import multiprocessing
from collections import OrderedDict

import gensim.models.doc2vec
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

from gensim.models.doc2vec import Doc2Vec

common_kwargs = dict(
    vector_size=100, epochs=20, min_count=2,
    sample=0, workers=multiprocessing.cpu_count(), negative=5, hs=0,
)

simple_models = [
    # PV-DBOW plain
    Doc2Vec(dm=0, **common_kwargs),
    # PV-DM w/ default averaging; a higher starting alpha may improve CBOW/PV-DM modes
    Doc2Vec(dm=1, window=10, alpha=0.05, comment='alpha=0.05', **common_kwargs),
    # PV-DM w/ concatenation - big, slow, experimental mode
    # window=5 (both sides) approximates paper's apparent 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, window=5, **common_kwargs),
]

for model in simple_models:
    model.build_vocab(alldocs)
    print(f"{model} vocabulary scanned & state initialized")

models_by_name = OrderedDict((str(model), model) for model in simple_models)

Doc2Vec<dbow,d100,n5,mc2,t16> vocabulary scanned & state initialized
Doc2Vec<dm/m,d100,n5,w10,mc2,t16> vocabulary scanned & state initialized
Doc2Vec<dm/c,d100,n5,w5,mc2,t16> vocabulary scanned & state initialized


In [10]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[1]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[2]])

In [11]:
import numpy as np
import statsmodels.api as sm
from random import sample

def logistic_predictor_from_data(train_targets, train_regressors):
    """Fit a statsmodel logistic predictor on supplied data"""
    logit = sm.Logit(train_targets, train_regressors)
    predictor = logit.fit(disp=0)
    # print(predictor.summary())
    return predictor

def error_rate_for_model(test_model, train_set, test_set):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets = [doc.sentiment for doc in train_set]
    train_regressors = [test_model.dv[doc.tags[0]] for doc in train_set]
    train_regressors = sm.add_constant(train_regressors)
    predictor = logistic_predictor_from_data(train_targets, train_regressors)

    test_regressors = [test_model.dv[doc.tags[0]] for doc in test_set]
    test_regressors = sm.add_constant(test_regressors)

    # Predict & evaluate
    test_predictions = predictor.predict(test_regressors)
    corrects = sum(np.rint(test_predictions) == [doc.sentiment for doc in test_set])
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), predictor)

In [12]:
from collections import defaultdict
error_rates = defaultdict(lambda: 1.0)  # To selectively print only best errors achieved

In [13]:
from random import shuffle
shuffled_alldocs = alldocs[:]
shuffle(shuffled_alldocs)

for model in simple_models:
    print(f"Training {model}")
    model.train(shuffled_alldocs, total_examples=len(shuffled_alldocs), epochs=model.epochs)

#    print(f"\nEvaluating {model}")
#    err_rate, err_count, test_count, predictor = error_rate_for_model(model, train_docs, test_docs)
#    error_rates[str(model)] = err_rate
#    print("\n%f %s\n" % (err_rate, model))

#for model in [models_by_name['dbow+dmm'], models_by_name['dbow+dmc']]:
#    print(f"\nEvaluating {model}")
#    err_rate, err_count, test_count, predictor = error_rate_for_model(model, train_docs, test_docs)
#    error_rates[str(model)] = err_rate
#    print(f"\n{err_rate} {model}\n")

Training Doc2Vec<dbow,d100,n5,mc2,t16>
Training Doc2Vec<dm/m,d100,n5,w10,mc2,t16>
Training Doc2Vec<dm/c,d100,n5,w5,mc2,t16>


In [14]:
simple_models[0].dv.most_similar(positive=[0, 1], negative=[39, 45])#

[(82, 0.10498181730508804),
 (92, 0.07196115702390671),
 (26, 0.06109962612390518),
 (49, 0.036519329994916916),
 (33, 0.030072033405303955),
 (41, 0.02534446120262146),
 (59, 0.025020528584718704),
 (46, 0.025019310414791107),
 (13, 0.022870682179927826),
 (52, 0.022708658128976822)]

In [40]:
vectors = []
for doc in alldocs:
    vectors.append(simple_models[2].infer_vector(doc.words))

In [41]:
vectors

[array([ 0.01912644,  0.01213147, -0.02747668,  0.01982494, -0.00770403,
         0.02001346, -0.02222989, -0.01522402, -0.02415604,  0.01618552,
        -0.01167114, -0.00949967,  0.04884406, -0.01124617, -0.01444912,
        -0.00755729,  0.00474329, -0.01405355, -0.02742491, -0.0033877 ,
        -0.0167539 , -0.00358507,  0.00096473,  0.00805479,  0.02944911,
         0.02472548, -0.01058561, -0.00410976, -0.00882334, -0.02171347,
        -0.01849627,  0.00403917,  0.0306354 ,  0.01261538, -0.01051473,
        -0.01659687, -0.02801268,  0.0243745 , -0.03129052,  0.02385901,
        -0.01227098, -0.00627255,  0.00878858, -0.00607363, -0.03141158,
        -0.01285943,  0.01624684,  0.01690485,  0.01248875, -0.00852586,
         0.00954568,  0.01452607, -0.01098487, -0.01153337, -0.02840109,
        -0.00066371,  0.00195186, -0.00605176, -0.04905409,  0.04184475,
        -0.00400232, -0.01054524,  0.00438607,  0.01231903, -0.02611953,
        -0.01520628,  0.01442017, -0.01657906,  0.0

In [42]:
joke_df = pd.DataFrame(vectors, index=np.arange(1, 101), columns=[f'joke_feature_{i}' for i in np.arange(1, 101)])
joke_df

,joke_feature_1,joke_feature_2,joke_feature_3,joke_feature_4,joke_feature_5,joke_feature_6,joke_feature_7,joke_feature_8,joke_feature_9,joke_feature_10,...,joke_feature_91,joke_feature_92,joke_feature_93,joke_feature_94,joke_feature_95,joke_feature_96,joke_feature_97,joke_feature_98,joke_feature_99,joke_feature_100
1,0.019126,0.012131,-0.027477,0.019825,-0.007704,0.020013,-0.022230,-0.015224,-0.024156,0.016186,...,0.013015,0.027021,-0.012433,0.006457,-0.014875,-0.015005,-0.018308,-0.001017,0.048864,0.007200
2,-0.011499,0.002285,0.003630,0.004051,0.003290,-0.000463,0.006562,-0.001884,0.015430,0.000577,...,-0.009672,-0.005080,-0.002575,-0.005094,0.006556,0.001787,0.013340,0.011139,-0.017809,-0.000959
3,-0.016668,-0.018199,0.012311,-0.010475,0.025088,0.000381,0.004331,0.023751,0.030754,-0.006464,...,-0.019020,-0.012687,-0.009148,-0.009461,0.019125,0.026595,0.013386,-0.013205,-0.019013,-0.015191
4,-0.032417,0.002634,0.027978,-0.019963,0.023566,0.007097,0.021878,0.044580,0.056586,-0.009916,...,-0.016706,-0.034643,0.006712,-0.007995,0.028974,0.030824,0.034675,0.001362,-0.046984,-0.017229
5,-0.014843,-0.046528,0.002575,-0.029940,0.030619,-0.011654,0.012663,0.048356,0.040379,-0.028446,...,-0.025209,-0.016499,-0.004959,-0.026612,0.021617,0.048349,0.002544,-0.042855,-0.021552,-0.023502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,-0.004857,0.012683,0.009205,-0.008275,-0.003818,0.006676,-0.003004,0.014902,0.007708,0.003922,...,0.011361,-0.006212,0.008240,-0.001582,0.010192,-0.002656,0.002402,0.000192,-0.013342,-0.000508
97,0.034768,0.040293,-0.008734,0.023405,-0.038429,0.024317,-0.039087,-0.039802,-0.081361,0.100287,...,0.036261,0.070944,-0.018116,0.070075,-0.064077,-0.088195,-0.061626,0.055512,0.110628,0.040841
98,0.017260,0.013419,-0.001536,-0.003390,-0.020430,-0.008701,0.003307,-0.016211,-0.019329,-0.022782,...,0.025377,-0.005499,0.053408,0.017966,0.011246,-0.036009,0.007395,0.013810,-0.027661,0.000457
99,0.052019,0.059732,-0.033075,0.046826,-0.088760,0.012471,-0.057181,-0.063283,-0.154123,0.091781,...,0.068348,0.117198,-0.028930,0.092582,-0.075235,-0.126480,-0.083787,0.035340,0.101882,0.086949


In [43]:
joke_df.to_csv('joke_doc2vec_vectors.csv', index=False)